In [3]:
import chess
import random
from IPython.display import clear_output
import time
import numpy as np

C = np.sqrt(2)

In [13]:
class Node:
    def __init__(self, board, parent=None):
        self.state = board
        self.utility = 0
        self.n_playthrough = 0
        self.parent = parent
        self.children = list()
        
    def UCB1(self):
        return self.utility/self.n_playthrough + C + np.sqrt(np.log(parent.n_playthrough)/self.n_playthrough)
    

In [11]:
class MonteCarloAgent:
    
    # main function for the Monte Carlo Tree Search
    def monte_carlo_tree_search(state):
        tree = Node(state)
        for i in range(100):
            leaf = select(tree)
            child = expand(leaf)
            result = simulate(child)
            if result.winner is state.turn:                
                back_prop(1, child)
            else:
                back_prop(0, child)

        return best_move(tree.state.legal_moves)
    
    def select(tree):
        return max(tree.children, key=lambda k: k.UCB1())
    
    def expand(leaf):
        for move in tree.state.legal_moves:
            leaf.state.push(move)
            leaf.children.append(Node(leaf.state.copy()))
            leaf.state.pop()
        return random.choice(leaf.children)
    
    def result(child):
        _board = child.state.copy()
        while not _child.state.is_game_over():
            move = random.choice(_board.legal_moves)
            _board.push(move)
        return _board.outcome()
        
    def backprop(result, child):
        if not child:
            return
        child.n_playthrough += 1
        child.utility += result
        backprop(result, child.parent)

In [6]:
?chess.Board

In [8]:
for i in range(10000000):
    a = 1